# CS 171 Final Project — Data Pre-processing Notebook

Builds a clean dataset that connects:

- Player-level performance statistics (games, goals, assists, minutes, cards, win rate) from `appearances.csv` and `club_games.csv`.
- Player event statistics (goals, cards, substitutions, shootout events) from `game_events.csv`.
- Latest player market value from `player_valuations.csv`.
- Player attributes (position, height, footedness, age) from `players.csv`.

We:

1. Load raw CSV files from the `archive` folder (from the Transfermarkt Kaggle dataset).
2. Aggregate per-player career stats and per-90-minute event rates.
3. Merge with player attributes and latest market value.
4. Build a numerical feature matrix `X` and target vector `y` (log market value).
5. Split into train / validation / test sets.
6. Standardize features using the training split.
7. Save arrays to `processed_player_value_dataset.npz` for the model notebook.


In [1]:
import os

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split

# for reproducibility
np.random.seed(42)


In [2]:
# Path to the unzipped Kaggle archive folder
data_dir = "archive"

players = pd.read_csv(os.path.join(data_dir, "players.csv"))
appearances = pd.read_csv(os.path.join(data_dir, "appearances.csv"))
player_vals = pd.read_csv(os.path.join(data_dir, "player_valuations.csv"))
games = pd.read_csv(os.path.join(data_dir, "games.csv"))
club_games = pd.read_csv(os.path.join(data_dir, "club_games.csv"))
game_events = pd.read_csv(os.path.join(data_dir, "game_events.csv"))

print("players:", players.shape)
print("appearances:", appearances.shape)
print("player_valuations:", player_vals.shape)
print("games:", games.shape)
print("club_games:", club_games.shape)
print("game_events:", game_events.shape)


players: (32601, 23)
appearances: (1706806, 13)
player_valuations: (496606, 5)
games: (74026, 23)
club_games: (148052, 11)
game_events: (1035043, 10)


In [3]:
print("players columns:", list(players.columns))
print("appearances columns:", list(appearances.columns))
print("player_valuations columns:", list(player_vals.columns))
print("games columns:", list(games.columns))
print("club_games columns:", list(club_games.columns))
print("game_events columns:", list(game_events.columns))


players columns: ['player_id', 'first_name', 'last_name', 'name', 'last_season', 'current_club_id', 'player_code', 'country_of_birth', 'city_of_birth', 'country_of_citizenship', 'date_of_birth', 'sub_position', 'position', 'foot', 'height_in_cm', 'contract_expiration_date', 'agent_name', 'image_url', 'url', 'current_club_domestic_competition_id', 'current_club_name', 'market_value_in_eur', 'highest_market_value_in_eur']
appearances columns: ['appearance_id', 'game_id', 'player_id', 'player_club_id', 'player_current_club_id', 'date', 'player_name', 'competition_id', 'yellow_cards', 'red_cards', 'goals', 'assists', 'minutes_played']
player_valuations columns: ['player_id', 'date', 'market_value_in_eur', 'current_club_id', 'player_club_domestic_competition_id']
games columns: ['game_id', 'competition_id', 'season', 'round', 'date', 'home_club_id', 'away_club_id', 'home_club_goals', 'away_club_goals', 'home_club_position', 'away_club_position', 'home_club_manager_name', 'away_club_manager_

## Aggregate per-player match statistics (from appearances + club_games)

We compute, for each player:

- `games_played`: number of distinct games.
- `total_minutes`: total minutes played.
- `total_goals`, `total_assists`.
- `total_yellow_cards`, `total_red_cards`.
- `wins`: number of games where the player's team won.
- `win_rate`: wins / games_played.


In [4]:
# Select relevant columns from appearances
appearances_small = appearances[[
    "game_id",
    "player_id",
    "player_club_id",
    "goals",
    "assists",
    "yellow_cards",
    "red_cards",
    "minutes_played"
]]

# Keep club and result info from club_games
club_games_small = club_games[["game_id", "club_id", "is_win"]]

# Merge to attach match result to each player-appearance
merged = appearances_small.merge(
    club_games_small,
    left_on=["game_id", "player_club_id"],
    right_on=["game_id", "club_id"],
    how="left"
)

# Missing is_win values are treated as False (no recorded win)
merged["is_win"] = merged["is_win"].fillna(False)

# Aggregate to per-player level
player_stats = merged.groupby("player_id").agg(
    games_played=("game_id", "nunique"),
    total_minutes=("minutes_played", "sum"),
    total_goals=("goals", "sum"),
    total_assists=("assists", "sum"),
    total_yellow_cards=("yellow_cards", "sum"),
    total_red_cards=("red_cards", "sum"),
    wins=("is_win", "sum")
).reset_index()

player_stats["win_rate"] = player_stats["wins"] / player_stats["games_played"]

print(player_stats.shape)
player_stats.head()


(25689, 9)


,player_id,games_played,total_minutes,total_goals,total_assists,total_yellow_cards,total_red_cards,wins,win_rate
0,10,136,8808,48,25,19,0,66,0.485294
1,26,152,13508,0,0,4,2,81,0.532895
2,65,122,8788,38,13,11,1,45,0.368852
3,77,4,307,0,0,0,0,3,0.750000
4,80,12,1080,0,0,0,0,10,0.833333


## Aggregate per-player event statistics (from game_events)

The `game_events.csv` table contains detailed match events with:

- `type`: one of `"Goals"`, `"Cards"`, `"Substitutions"`, `"Shootout"`.
- `player_id`: the main player involved in the event.
- `player_in_id`: for substitution events, the player entering the field.

We create per-player counts and per-90-minute rates for:

- `events_goals`
- `events_cards`
- `events_subs_out` (player substituted out)
- `events_subs_in` (player substituted in)
- `events_shootout`


In [5]:
# Parse dates 
game_events["date"] = pd.to_datetime(game_events["date"], errors="coerce")

# Base counts by player_id and event type
base_events = game_events.groupby(["player_id", "type"]).size().unstack(fill_value=0)

# Counts of substitutions where the player entered the field (player_in_id)
subs_in = game_events[~game_events["player_in_id"].isna()]
subs_in_counts = subs_in.groupby("player_in_id").size()

# Build a per-player event table
event_stats = base_events.copy()
event_stats = event_stats.rename(columns={
    "Goals": "events_goals",
    "Cards": "events_cards",
    "Substitutions": "events_subs_out",
    "Shootout": "events_shootout"
})
event_stats["events_subs_in"] = subs_in_counts
event_stats = event_stats.fillna(0).reset_index()

event_stats.head()


type,player_id,events_cards,events_goals,events_shootout,events_subs_out,events_subs_in
0,10,19,48,0,52,37.0
1,26,6,0,0,0,2.0
2,65,12,38,0,35,26.0
3,77,0,0,0,1,1.0
4,109,4,1,0,3,0.0


In [6]:
# Merge event stats into player_stats
player_stats_ev = player_stats.merge(event_stats, on="player_id", how="left")

# Replace missing event counts (players with no recorded events) with zero
for col in ["events_cards", "events_goals", "events_subs_out", "events_subs_in", "events_shootout"]:
    player_stats_ev[col] = player_stats_ev[col].fillna(0)

print(player_stats_ev.shape)
player_stats_ev.head()


(25689, 14)


,player_id,games_played,total_minutes,total_goals,total_assists,total_yellow_cards,total_red_cards,wins,win_rate,events_cards,events_goals,events_shootout,events_subs_out,events_subs_in
0,10,136,8808,48,25,19,0,66,0.485294,19.0,48.0,0.0,52.0,37.0
1,26,152,13508,0,0,4,2,81,0.532895,6.0,0.0,0.0,0.0,2.0
2,65,122,8788,38,13,11,1,45,0.368852,12.0,38.0,0.0,35.0,26.0
3,77,4,307,0,0,0,0,3,0.750000,0.0,0.0,0.0,1.0,1.0
4,80,12,1080,0,0,0,0,10,0.833333,0.0,0.0,0.0,0.0,0.0


In [7]:
# Per-90-minute event rates
player_stats_ev["minutes_per_90"] = player_stats_ev["total_minutes"] / 90.0
minutes_per_90 = player_stats_ev["minutes_per_90"].replace(0, np.nan)

for col in ["events_cards", "events_goals", "events_subs_out", "events_subs_in", "events_shootout"]:
    per90_col = col + "_per90"
    player_stats_ev[per90_col] = player_stats_ev[col] / minutes_per_90
    player_stats_ev[per90_col] = player_stats_ev[per90_col].fillna(0)

player_stats_ev[[
    "player_id",
    "games_played",
    "total_minutes",
    "events_goals_per90",
    "events_cards_per90"
]].head()


,player_id,games_played,total_minutes,events_goals_per90,events_cards_per90
0,10,136,8808,0.490463,0.194142
1,26,152,13508,0.000000,0.039976
2,65,122,8788,0.389167,0.122895
3,77,4,307,0.000000,0.000000
4,80,12,1080,0.000000,0.000000


## Merge with latest market value and player attributes

We:

1. Take the last available market value per player from `player_valuations.csv`.
2. Merge with `player_stats_ev`.
3. Add player attributes (position, height, footedness, date of birth) from `players.csv`.
4. Compute age at the time of the last valuation.


In [8]:
# Latest market value per player
player_vals["date"] = pd.to_datetime(player_vals["date"], errors="coerce")
player_vals_sorted = player_vals.sort_values(["player_id", "date"])

last_vals = player_vals_sorted.groupby("player_id").tail(1)[
    ["player_id", "date", "market_value_in_eur"]
].rename(columns={"date": "last_valuation_date"})

# Merge stats with value
full_stats = player_stats_ev.merge(last_vals, on="player_id", how="inner")
print(full_stats.shape)
full_stats.head()


(25324, 22)


,player_id,games_played,total_minutes,total_goals,total_assists,total_yellow_cards,total_red_cards,wins,win_rate,events_cards,...,events_subs_out,events_subs_in,minutes_per_90,events_cards_per90,events_goals_per90,events_subs_out_per90,events_subs_in_per90,events_shootout_per90,last_valuation_date,market_value_in_eur
0,10,136,8808,48,25,19,0,66,0.485294,19.0,...,52.0,37.0,97.866667,0.194142,0.490463,0.531335,0.378065,0.0,2016-01-04,1000000
1,26,152,13508,0,0,4,2,81,0.532895,6.0,...,0.0,2.0,150.088889,0.039976,0.000000,0.000000,0.013325,0.0,2017-12-28,750000
2,65,122,8788,38,13,11,1,45,0.368852,12.0,...,35.0,26.0,97.644444,0.122895,0.389167,0.358443,0.266272,0.0,2016-06-21,1000000
3,77,4,307,0,0,0,0,3,0.750000,0.0,...,1.0,1.0,3.411111,0.000000,0.000000,0.293160,0.293160,0.0,2016-11-15,200000
4,80,12,1080,0,0,0,0,10,0.833333,0.0,...,0.0,0.0,12.000000,0.000000,0.000000,0.000000,0.000000,0.0,2018-06-05,100000


In [9]:
# Player attributes
players_small = players[[
    "player_id",
    "position",
    "sub_position",
    "height_in_cm",
    "foot",
    "country_of_citizenship",
    "date_of_birth"
]].copy()

players_small["date_of_birth"] = pd.to_datetime(players_small["date_of_birth"], errors="coerce")

full_df = full_stats.merge(players_small, on="player_id", how="left")

full_df["last_valuation_date"] = pd.to_datetime(full_df["last_valuation_date"], errors="coerce")

# Age in years at last valuation
full_df["age_years"] = (
    full_df["last_valuation_date"] - full_df["date_of_birth"]
).dt.days / 365.25

print(full_df.shape)
full_df[[
    "player_id",
    "games_played",
    "total_minutes",
    "events_goals_per90",
    "age_years",
    "market_value_in_eur"
]].head()


(25324, 29)


,player_id,games_played,total_minutes,events_goals_per90,age_years,market_value_in_eur
0,10,136,8808,0.490463,37.571526,1000000
1,26,152,13508,0.000000,37.393566,750000
2,65,122,8788,0.389167,35.389459,1000000
3,77,4,307,0.000000,38.524298,200000
4,80,12,1080,0.000000,37.215606,100000


## Filter to players with enough data and valid market values

We keep only players who:

- Have played at least `min_games` matches (to make stats meaningful).
- Have a positive, non-missing `market_value_in_eur`.


In [10]:
min_games = 5

mask_games = full_df["games_played"] >= min_games
mask_value = full_df["market_value_in_eur"].notna() & (full_df["market_value_in_eur"] > 0)

clean_df = full_df[mask_games & mask_value].copy()

print("original players:", full_df.shape[0])
print("after filtering:", clean_df.shape[0])


original players: 25324
after filtering: 20883


## Handle missing values and define features/target

- Fill missing numeric values with the median of that column.
- Fill missing categorical values with `"Unknown"`.
- Target: `log_market_value = log(market_value_in_eur)`.
- Features:
    - Career aggregates: games, minutes, goals, assists, cards, wins, win_rate, height, age.
    - Per-90-minute event rates: goals, cards, substitutions, shootout events.
    - One-hot encodings for position and foot.


In [11]:
# Numeric columns that might contain missing values
numeric_cols = [
    "total_minutes",
    "total_goals",
    "total_assists",
    "total_yellow_cards",
    "total_red_cards",
    "games_played",
    "wins",
    "win_rate",
    "height_in_cm",
    "age_years",
    "events_cards_per90",
    "events_goals_per90",
    "events_subs_out_per90",
    "events_subs_in_per90",
    "events_shootout_per90"
]

for col in numeric_cols:
    if clean_df[col].isna().any():
        clean_df[col] = clean_df[col].fillna(clean_df[col].median())

# Simple categorical imputation
clean_df["position"] = clean_df["position"].fillna("Unknown")
clean_df["sub_position"] = clean_df["sub_position"].fillna("Unknown")
clean_df["foot"] = clean_df["foot"].fillna("Unknown")

# Target: log market value
clean_df["log_market_value"] = np.log(clean_df["market_value_in_eur"])

# Base numeric features
base_feature_cols = [
    "games_played",
    "total_minutes",
    "total_goals",
    "total_assists",
    "total_yellow_cards",
    "total_red_cards",
    "wins",
    "win_rate",
    "height_in_cm",
    "age_years",
    "events_cards_per90",
    "events_goals_per90",
    "events_subs_out_per90",
    "events_subs_in_per90",
    "events_shootout_per90"
]

# One-hot encode position and dominant foot
df_encoded = pd.get_dummies(
    clean_df,
    columns=["position", "foot"],
    drop_first=True
)

feature_cols = base_feature_cols + [
    col for col in df_encoded.columns
    if col.startswith("position_") or col.startswith("foot_")
]

print("Number of features:", len(feature_cols))
feature_cols


Number of features: 22


['games_played',
 'total_minutes',
 'total_goals',
 'total_assists',
 'total_yellow_cards',
 'total_red_cards',
 'wins',
 'win_rate',
 'height_in_cm',
 'age_years',
 'events_cards_per90',
 'events_goals_per90',
 'events_subs_out_per90',
 'events_subs_in_per90',
 'events_shootout_per90',
 'position_Defender',
 'position_Goalkeeper',
 'position_Midfield',
 'position_Missing',
 'foot_both',
 'foot_left',
 'foot_right']

In [12]:
# Build X, y arrays
X_all = df_encoded[feature_cols].values.astype(np.float32)
y_all = df_encoded["log_market_value"].values.astype(np.float32)

print("X_all shape:", X_all.shape)
print("y_all shape:", y_all.shape)


X_all shape: (20883, 22)
y_all shape: (20883,)


## Train / validation / test split

We split the data as:

- 60% training
- 20% validation
- 20% test

using a fixed random seed for reproducibility.


In [13]:
# First split off a test set (20%)
X_temp, X_test, y_temp, y_test = train_test_split(
    X_all,
    y_all,
    test_size=0.2,
    random_state=42
)

# Then split remaining into train (60%) and val (20%)
X_train, X_val, y_train, y_val = train_test_split(
    X_temp,
    y_temp,
    test_size=0.25,  # 0.25 of 0.8 = 0.2 of original
    random_state=42
)

print("Train:", X_train.shape, y_train.shape)
print("Val:", X_val.shape, y_val.shape)
print("Test:", X_test.shape, y_test.shape)


Train: (12529, 22) (12529,)
Val: (4177, 22) (4177,)
Test: (4177, 22) (4177,)


## Standardize features using training statistics

We compute the mean and standard deviation of each feature on the training split only,
then apply the same transformation to validation and test features:

$$
X_{scaled} = \frac{X - \mu_{train}}{\sigma_{train}}
$$


In [14]:
# Compute mean and std on training set
X_mean = X_train.mean(axis=0)
X_std = X_train.std(axis=0)

# Avoid division by zero
X_std[X_std == 0] = 1.0

X_train_scaled = (X_train - X_mean) / X_std
X_val_scaled = (X_val - X_mean) / X_std
X_test_scaled = (X_test - X_mean) / X_std

print("X_train_scaled mean (approx):", X_train_scaled.mean(axis=0)[:5])
print("X_train_scaled std (approx):", X_train_scaled.std(axis=0)[:5])


X_train_scaled mean (approx): [ 1.1824831e-07  1.1254903e-06 -1.5090741e-07 -2.3753371e-08
  3.8439264e-08]
X_train_scaled std (approx): [0.99999523 0.9999992  0.9999767  0.999985   1.0000142 ]


## Save processed arrays for the model notebook

We save:

- `X_train`, `X_val`, `X_test`
- `y_train`, `y_val`, `y_test`
- `feature_names`
- `X_mean`, `X_std`

into a single `.npz` file `processed_player_value_dataset.npz` in the current folder.


In [16]:
out_path = "processed_player_value_dataset.npz"

np.savez(
    out_path,
    X_train=X_train_scaled,
    X_val=X_val_scaled,
    X_test=X_test_scaled,
    y_train=y_train,
    y_val=y_val,
    y_test=y_test,
    feature_names=np.array(feature_cols),
    X_mean=X_mean,
    X_std=X_std
)

out_path


'processed_player_value_dataset.npz'